In [1]:
import time
import requests
import json
import numpy as np

from jupyter_dash import JupyterDash
import dash
import dash_daq as daq
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output

In [2]:
IP = '192.168.1.11'
REFRESH_TIME = 3000

with open('moisture_intervals.json', 'r') as f:
    MOISTURE_INTERVALS = json.load(f)

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])
server = app.server

In [4]:
temperature_card = dbc.Card([
    dbc.CardBody([
        html.H6('Temperature'),
        html.H4([html.Span('-', id='temperature_value'), ' °C'],
                className='text-center'),
    ]),
])

humidity_card = dbc.Card([
    dbc.CardBody([
        html.H6('Humidity'),
        html.H4([html.Span('-', id='humidity_value'), ' RH'],
                className='text-center'),
    ])
])

moisture_card = dbc.Card([
    dbc.CardBody([
        html.H6('Moisture'),
        daq.GraduatedBar(
            id='moisture_level',
            max=max(MOISTURE_INTERVALS.values(),
                    key=(lambda x: x[1]))[1],
            showCurrentValue=True,
            step=100,
            color={'ranges': MOISTURE_INTERVALS,
                   'gradient': True},
            style={'display': 'flex', 'justifyContent': 'center'}
        ),
    ])
])

measurements_cards = dbc.CardDeck([temperature_card, humidity_card, moisture_card])

In [5]:
app.layout = html.Div(
    [
        dcc.Interval(id='interval', interval=REFRESH_TIME),
        measurements_cards,
    ],
    style={'padding-left': '15%',
           'padding-right': '15%'})

In [6]:
@app.callback(
    Output('temperature_value', 'children'),
    Output('humidity_value', 'children'),
    Output('moisture_level', 'value'),
    Input('interval', 'n_intervals')
)
def read_sensors(n_clicks):
    # TODO: in case of any sensor returns None still update values for other sensors
    for _ in range(10):
        sensors_data = requests.get('http://{}/read'.format(IP),
                                params={'n':5, 't':10})
    
        if sensors_data.status_code != 200:
            continue
        
        values = sensors_data.json()
        if any(None in v for v in values.values()):
            continue
        
        avg_values = {k: np.mean(v) for k, v in values.items()}
        if np.isnan(list(avg_values.values())).any():
            continue
            
        return [avg_values[k] for k in ['temperature', 'humidity', 'moisture']]
    
    return ['-'] * 3

In [7]:
app.run_server(mode='inline', debug=True)

In [8]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
